In [12]:
from transformers import AutoConfig

In [13]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained("openai-community/gpt2")

In [14]:
config.n_layer

12

In [15]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel(config)

In [16]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [21]:
model.transformer.h[0]

GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [ ]:
from safetensors.torch import load_file

state_dict = load_file("../../../data/model.safetensors")

In [24]:
from safetensors import safe_open

stage_sd = {}
owm_key = [model.transformer.h[0], model.transformer.h[1], model.transformer.ln_f]
with safe_open("../../../data/model.safetensors", framework="pt") as f:
    for k in f.keys():
        print(k)
        # if owns_key(k):
        #     stage_sd[k] = f.get_tensor(k)

h.0.attn.bias
h.0.attn.c_attn.bias
h.0.attn.c_attn.weight
h.0.attn.c_proj.bias
h.0.attn.c_proj.weight
h.0.ln_1.bias
h.0.ln_1.weight
h.0.ln_2.bias
h.0.ln_2.weight
h.0.mlp.c_fc.bias
h.0.mlp.c_fc.weight
h.0.mlp.c_proj.bias
h.0.mlp.c_proj.weight
h.1.attn.bias
h.1.attn.c_attn.bias
h.1.attn.c_attn.weight
h.1.attn.c_proj.bias
h.1.attn.c_proj.weight
h.1.ln_1.bias
h.1.ln_1.weight
h.1.ln_2.bias
h.1.ln_2.weight
h.1.mlp.c_fc.bias
h.1.mlp.c_fc.weight
h.1.mlp.c_proj.bias
h.1.mlp.c_proj.weight
h.10.attn.bias
h.10.attn.c_attn.bias
h.10.attn.c_attn.weight
h.10.attn.c_proj.bias
h.10.attn.c_proj.weight
h.10.ln_1.bias
h.10.ln_1.weight
h.10.ln_2.bias
h.10.ln_2.weight
h.10.mlp.c_fc.bias
h.10.mlp.c_fc.weight
h.10.mlp.c_proj.bias
h.10.mlp.c_proj.weight
h.11.attn.bias
h.11.attn.c_attn.bias
h.11.attn.c_attn.weight
h.11.attn.c_proj.bias
h.11.attn.c_proj.weight
h.11.ln_1.bias
h.11.ln_1.weight
h.11.ln_2.bias
h.11.ln_2.weight
h.11.mlp.c_fc.bias
h.11.mlp.c_fc.weight
h.11.mlp.c_proj.bias
h.11.mlp.c_proj.weight
h.2.at

In [25]:
layers = []
for layer in model.transformer.h:
            layers.append(layer)  

In [28]:
layers

[GPT2Block(
   (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (attn): GPT2Attention(
     (c_attn): Conv1D(nf=2304, nx=768)
     (c_proj): Conv1D(nf=768, nx=768)
     (attn_dropout): Dropout(p=0.1, inplace=False)
     (resid_dropout): Dropout(p=0.1, inplace=False)
   )
   (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (mlp): GPT2MLP(
     (c_fc): Conv1D(nf=3072, nx=768)
     (c_proj): Conv1D(nf=768, nx=3072)
     (act): NewGELUActivation()
     (dropout): Dropout(p=0.1, inplace=False)
   )
 ),
 GPT2Block(
   (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (attn): GPT2Attention(
     (c_attn): Conv1D(nf=2304, nx=768)
     (c_proj): Conv1D(nf=768, nx=768)
     (attn_dropout): Dropout(p=0.1, inplace=False)
     (resid_dropout): Dropout(p=0.1, inplace=False)
   )
   (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (mlp): GPT2MLP(
     (c_fc): Conv1D(nf=3072, nx=768)
     (c_proj): Conv1D(nf=768, nx=3072)
     (act): NewGE

In [30]:
layers_per_node = 6
local_rank = 0
out_layers = layers[layers_per_node * local_rank : ((layers_per_node * local_rank) + layers_per_node) - 1]

TypeError: list indices must be integers or slices, not str

In [ ]:
from transformers import AutoConfig
from typing import List
import torch

def get_layers_per_node(config: AutoConfig, model, num_nodes: int, local_rank: int, model_name: str) -> List:
    
    out_layers = {}
    
    if model_name == 'causal_gpt2':
        
        total_layers = config.n_layer
        
        layers_per_node = total_layers // num_nodes
        
        layers = []
        
        for layer in model.transformer.h:
            layers.append(layer)     
            
        if local_rank == 0:
            out_layers['model.transformer.wte'] = model.transformer.wte
            out_layers['model.transformer.wpe'] = model.transformer.wpe
   
        
        elif local_rank == num_nodes - 1:
            selected_layers = layers[layers_per_node * local_rank : ((layers_per_node * local_rank) + layers_per_node) - 1]
            
            for layer in selected_layers:
                out_layers[f'model.transformer.h.{layers.index(layer)}'] = layer
                
            out_layers['model.transformer.ln_f'] = model.transformer.ln_f   
        
        
        else:
            selected_layers = layers[layers_per_node * local_rank : ((layers_per_node * local_rank) + layers_per_node) - 1]
            
            for layer in selected_layers:
                out_layers[f'model.transformer.h.{layers.index(layer)}'] = layer
        
        return out_layers
    
    

        
        

In [36]:
out_layers = get_layers_per_node(config, model, num_nodes=4, local_rank=3, model_name='causal_gpt2')

In [65]:
out_layers

results = []

In [66]:
for i in out_layers['model.transformer.h.10'].named_parameters():
    results.append('h.10.' + i[0])

In [67]:
results

['h.10.ln_1.weight',
 'h.10.ln_1.bias',
 'h.10.attn.c_attn.weight',
 'h.10.attn.c_attn.bias',
 'h.10.attn.c_proj.weight',
 'h.10.attn.c_proj.bias',
 'h.10.ln_2.weight',
 'h.10.ln_2.bias',
 'h.10.mlp.c_fc.weight',
 'h.10.mlp.c_fc.bias',
 'h.10.mlp.c_proj.weight',
 'h.10.mlp.c_proj.bias']

In [69]:
from safetensors import safe_open

stage_sd = {}
with safe_open("../../../data/model.safetensors", framework="pt") as f:
    for k in f.keys():
        
        if k in results:
            stage_sd[k] = f.get_tensor(k)
        #     stage_sd[k] = f.get_tensor(k)

In [70]:
stage_sd

{'h.10.attn.c_attn.bias': tensor([-0.0301,  0.1360, -0.3842,  ..., -0.0599,  0.1059,  0.0276]),
 'h.10.attn.c_attn.weight': tensor([[ 0.2200,  0.2637,  0.0422,  ...,  0.1725,  0.3419,  0.0662],
         [-0.1269, -0.0215, -0.0951,  ..., -0.0543, -0.2407, -0.0239],
         [ 0.0709, -0.0726, -0.0253,  ...,  0.2005, -0.0454,  0.0693],
         ...,
         [ 0.0642, -0.0317,  0.1332,  ..., -0.1037,  0.2958,  0.0867],
         [-0.1892, -0.1039,  0.0752,  ..., -0.0223,  0.0835, -0.0417],
         [ 0.1380,  0.1194, -0.1244,  ..., -0.2325, -0.0402, -0.0117]]),
 'h.10.attn.c_proj.bias': tensor([-1.2379e-02,  1.4348e-02, -4.4527e-03, -4.5073e-02, -1.2405e-02,
          8.0911e-02,  1.5559e-01,  5.4038e-02,  6.7509e-02,  3.0687e-02,
          3.0247e-01, -9.8182e-03,  8.1174e-02, -1.7205e-01,  5.2295e-02,
          1.3546e-01, -2.1004e-01, -2.1952e-01,  4.9703e-02,  2.3138e-02,
          2.2932e-02,  2.2249e-03, -7.0906e-02, -3.5156e-02,  2.0411e-01,
         -1.2486e-01,  1.3384e-01, -1.92